In [1]:
import sys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth
import pandas as pd
import os

In [ ]:


#-------------------------------------------------------Driver CONFIGURATION-------------------------------------------------------------------------#
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-geolocation")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-extensions")

driver = webdriver.Chrome(options=chrome_options)
stealth(driver,
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True)


url = "https://www.bestbuy.com/?intl=nosplash"
class_search_bar = "search-input"
class_search_button = "header-search-button"

class_items = "sku-item"

def handle_survey():
    try:
        # If survey is noticed then click the no button, else, continue
        no_thanks_button = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.ID, "survey_invite_no"))
        )
        no_thanks_button.click()
        print("Survey dismissed")
    except:
        print("No survey popup")
        
next_page = None
links = []
products_data = []
all_headers = ["Link", "Name", 'Guides'] 
        

def process_product(driver, link):
    global products_data, all_headers
    driver.get(link)
    driver.implicitly_wait(20)
    
    #----------------------------------------------------------------------------------------------------------------------------------------------------------------------GET LINK
    product_info = {'Link': link}
    try:
        product_name_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "h1"))
        )
        product_info['Name'] = product_name_element.text
    except Exception as e:
        product_info['Name'] = "N/A"
        print("Error getting product name:", e)
    if "Package" in product_info['Name']: return 
    
    #=========================================================================TESTE=====================================================
    try:   
        class_manufacturer_btn = "c-button-unstyled.manufacturer-content-drawer-btn.w-full.flex.justify-content-between.align-items-center.py-200"
        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME,class_manufacturer_btn))).click()
        
        try:
            # Aguardar até que o iFrame esteja disponível e mudar para ele
            WebDriverWait(driver, 30).until(EC.frame_to_be_available_and_switch_to_it((By.CLASS_NAME, 'mt-50.w-full.h-full')))
            time.sleep(2)  # Espera adicional para garantir o carregamento do conteúdo

            # Executar JavaScript para rolar dentro do iframe até o final
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Espera para garantir que todos os elementos sejam carregados

            # Voltar para o conteúdo principal e rolar a div específica
            driver.switch_to.default_content()
            scroll_div = driver.find_element(By.CLASS_NAME, 'overflow-scroll-wrapper.v-scrolling-vertical.flex-grow-1')
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scroll_div)
            time.sleep(5)  # Espera para carregar o conteúdo adicional

            # Voltar para o iframe para continuar a rolagem, se necessário
            driver.switch_to.frame(driver.find_element(By.CLASS_NAME, 'mt-50.w-full.h-full'))
            driver.execute_script("window.scrollBy(0, 1000);")
            driver.execute_script("window.scrollBy(0, 500);")  
            driver.execute_script("window.scrollBy(0, 1000);")
            driver.execute_script("window.scrollBy(0, 250);")
            driver.execute_script("window.scrollBy(0, 500);")
            driver.execute_script("window.scrollBy(0, 250);")      
            driver.execute_script("window.scrollBy(0, 1000);")
            driver.execute_script("window.scrollBy(0, 1000);")
            driver.execute_script("window.scrollBy(0, 1000);")
            driver.execute_script("window.scrollBy(0, 1000);")
            driver.execute_script("window.scrollBy(0, 1000);")
            
            
            div_I = driver.find_element(By.CLASS_NAME,'syndi_powerpage')
            div_II = div_I.find_element(By.CLASS_NAME,'syndigo-powerpage-grid')
            divs = div_II.find_elements(By.CLASS_NAME,'syndigo-widget-section-header')
            for div in divs:
                try:
                    section = div.find_element(By.CLASS_NAME,'syndigo-documentgallery')
                except:
                    pass
            if section == None:
                print('Error findind the section')
            else:
                try:
                    guides_ul = section.find_element(By.CLASS_NAME,"syndigo-documentgallery-tiles")
                    guides_lis = guides_ul.find_elements(By.CLASS_NAME,'syndigo-documentgallery-tiles')
                    
                    try:
                        for li in guides_lis:
                            try:
                                div = li.find_element(By.TAG_NAME,'div')
                                guide_title = div.find_element(By.TAG_NAME,'h4').text
                                try:
                                    div = div.find_element(By.TAG_NAME,'div')
                                    guide_download_link = div.find_element(By.CLASS_NAME, 'syndigo-documentgallery-download-link')
                                    product_info[guide_title] = guide_download_link.get_attribute('href')
                            
                                    if guide_title not in all_headers:
                                        all_headers.append(guide_title)
                                        
                                except Exception as e: print("Error getting guides Link: ", e)        
                            except Exception as e: print("Error getting guides Titles: ", e)    
                    except Exception as e: print("Error getting guides: ", e)
                except Exception as e: print("Error getting guides list: ", e)
                            
            driver.switch_to.default_content()
            
            try:
                WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CLASS_NAME,'c-close-icon'))).click()
            except:
                driver.refresh()   
        except Exception as e: print("Error finding iFrame: ",e)     
    except Exception as e: print("Error clicking in the manufacturer button: ", e)
        
    #========================================================================FIM DA ABA DE TESTES======================================
    products_data.append(product_info)
    
    
def process_products(driver):
    global links
    for link in links: 
        process_product(driver, link)
    links.clear()
    
#---------------------------------------------------------------------------Begining---------------------------------------------------------------------#    
#--------------------------------------------------------------------------------------------------------------------------------------------------------#    
try:        
    driver.get(url)
    handle_survey()
    driver.implicitly_wait(20)  # Wait for it to load
    print("Page loaded.")
    search = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, class_search_bar)))
    search.send_keys("a")
    
    time.sleep(2)
    
    button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.CLASS_NAME, class_search_button)))
    button.click()
    
    driver.implicitly_wait(20)  # Wait for it to load
    
    '''
    This section is useful for test purposes
        You need to uncomment and fix indentation, also look at the code below, might be useful
    '''
    test_links = "statics/test_product_link_BB.csv"
    test_output_path = 'outputs/Best_Buy/test_product_data.csv'
    
    #If exists, run based on the links given
    links = pd.read_csv(test_links)
    links = links["Product Links"].to_list()
    
except Exception as e:
    print("Not able to run the code, error: ", e)

if not os.path.exists(test_links):
    df = pd.DataFrame(links, columns=['Product Links'])
    df.to_csv(test_links, index=False)

process_products(driver)

# Convert the list of dictionaries into a dataframe
df = pd.DataFrame(products_data)

# Ensure all headers are included
for header in all_headers:
    if header not in df.columns:
        df[header] = "N/A"

# Reorder columns based on all_headers
columns_order = all_headers + sorted([header for header in all_headers if header not in all_headers])
df = df.reindex(columns=columns_order)

print(df.head(20))
df.to_csv(test_output_path, index=False)

driver.quit()

# Different Test


In [6]:


#-------------------------------------------------------Driver CONFIGURATION-------------------------------------------------------------------------#
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-geolocation")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-extensions")

driver = webdriver.Chrome(options=chrome_options)
stealth(driver,
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True)


url = "https://www.bestbuy.com/?intl=nosplash"
class_search_bar = "search-input"
class_search_button = "header-search-button"

class_items = "sku-item"

def handle_survey():
    try:
        # If survey is noticed then click the no button, else, continue
        no_thanks_button = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.ID, "survey_invite_no"))
        )
        no_thanks_button.click()
        print("Survey dismissed")
    except:
        print("No survey popup")
        
next_page = None
links = []
products_data = []
all_headers = ["Link", "Name", 'Guides'] 
        

def process_product(driver, link):
    global products_data, all_headers
    driver.get(link)
    driver.implicitly_wait(20)
    
    #----------------------------------------------------------------------------------------------------------------------------------------------------------------------GET LINK
    product_info = {'Link': link}
    try:
        product_name_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "h1"))
        )
        product_info['Name'] = product_name_element.text
    except Exception as e:
        product_info['Name'] = "N/A"
        print("Error getting product name:", e)
    if "Package" in product_info['Name']: return 
    
    #=========================================================================TESTE=====================================================
    try:
        # Localizar e clicar no botão do fabricante
        class_manufacturer_btn = "c-button-unstyled.manufacturer-content-drawer-btn.w-full.flex.justify-content-between.align-items-center.py-200"
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, class_manufacturer_btn))).click()

        try:
            # Navegar para o link do iframe
            print(WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="pdp-drawer-overlay-backdrop"]/div/div/div[2]/iframe'))).get_attribute("src"))
            driver.get(WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="pdp-drawer-overlay-backdrop"]/div/div/div[2]/iframe'))))
            
            try:
                # Encontrar todos os elementos <li> com documentos
                doc_items = driver.find_elements(By.CSS_SELECTOR, "ul.syndigo-documentgallery-tiles li")
                for item in doc_items:
                    try:
                        # Encontrar o link para download
                        guide_title = item.find_element(By.CSS_SELECTOR,'h4.syndigo-documentgallery-title').text
                        download_link = item.find_element(By.CSS_SELECTOR, "a.syndigo-documentgallery-download-link")
                        link = download_link.get_attribute("href")
                        product_info[guide_title] = link
                        
                        if guide_title not in all_headers: all_headers.append(guide_title)                        

                        print(f"Download link: {link}")
                    except Exception as e:
                        print(f"Erro ao extrair link de download: {e}")
            except Exception as e:
                print(f"Erro ao encontrar elementos de documentos: {e}")
            finally:
                driver.get(link)
                driver.implicitly_wait(20)
        except Exception as e:
         print("Error handling iframe: ", e)     
    except Exception as e:
        print("Error clicking in the manufacturer button: ", e)
        
    #========================================================================FIM DA ABA DE TESTES======================================
    products_data.append(product_info)
    
    
def process_products(driver):
    global links
    for link in links: 
        process_product(driver, link)
    links.clear()
    
#---------------------------------------------------------------------------Begining---------------------------------------------------------------------#    
#--------------------------------------------------------------------------------------------------------------------------------------------------------#    
try:        
    driver.get(url)
    handle_survey()
    driver.implicitly_wait(20)  # Wait for it to load
    print("Page loaded.")
    search = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, class_search_bar)))
    search.send_keys("a")
    
    time.sleep(2)
    
    button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.CLASS_NAME, class_search_button)))
    button.click()
    
    driver.implicitly_wait(20)  # Wait for it to load
    
    '''
    This section is useful for test purposes
        You need to uncomment and fix indentation, also look at the code below, might be useful
    '''
    test_links = "statics/test_product_link_BB.csv"
    test_output_path = 'outputs/Best_Buy/test_product_data.csv'
    
    #If exists, run based on the links given
    links = pd.read_csv(test_links)
    links = links["Product Links"].to_list()
    
except Exception as e:
    print("Not able to run the code, error: ", e)

if not os.path.exists(test_links):
    df = pd.DataFrame(links, columns=['Product Links'])
    df.to_csv(test_links, index=False)

process_products(driver)

# Convert the list of dictionaries into a dataframe
df = pd.DataFrame(products_data)

# Ensure all headers are included
for header in all_headers:
    if header not in df.columns:
        df[header] = "N/A"

# Reorder columns based on all_headers
columns_order = all_headers + sorted([header for header in all_headers if header not in all_headers])
df = df.reindex(columns=columns_order)

print(df.head(20))
df.to_csv(test_output_path, index=False)

driver.quit()

No survey popup
Page loaded.

Error handling iframe:  Message: invalid argument: 'url' must be a string
  (Session info: chrome=127.0.6533.73)
Stacktrace:
	GetHandleVerifier [0x00007FF767BE9632+30946]
	(No symbol) [0x00007FF767B9E3C9]
	(No symbol) [0x00007FF767A96FDA]
	(No symbol) [0x00007FF767B2BAF4]
	(No symbol) [0x00007FF767B0CA6A]
	(No symbol) [0x00007FF767B2B041]
	(No symbol) [0x00007FF767B0C813]
	(No symbol) [0x00007FF767ADA6E5]
	(No symbol) [0x00007FF767ADB021]
	GetHandleVerifier [0x00007FF767D1F83D+1301229]
	GetHandleVerifier [0x00007FF767D2BDB7+1351783]
	GetHandleVerifier [0x00007FF767D22A03+1313971]
	GetHandleVerifier [0x00007FF767C1DD06+245686]
	(No symbol) [0x00007FF767BA758F]
	(No symbol) [0x00007FF767BA3804]
	(No symbol) [0x00007FF767BA3992]
	(No symbol) [0x00007FF767B9A3EF]
	BaseThreadInitThunk [0x00007FFA94047344+20]
	RtlUserThreadStart [0x00007FFA95B1CC91+33]


Error handling iframe:  Message: invalid argument: 'url' must be a string
  (Session info: chrome=127.0.6533.

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.73)
Stacktrace:
	GetHandleVerifier [0x00007FF767BE9632+30946]
	(No symbol) [0x00007FF767B9E3C9]
	(No symbol) [0x00007FF767A96FDA]
	(No symbol) [0x00007FF767A6CB85]
	(No symbol) [0x00007FF767B137A7]
	(No symbol) [0x00007FF767B2A771]
	(No symbol) [0x00007FF767B0C813]
	(No symbol) [0x00007FF767ADA6E5]
	(No symbol) [0x00007FF767ADB021]
	GetHandleVerifier [0x00007FF767D1F83D+1301229]
	GetHandleVerifier [0x00007FF767D2BDB7+1351783]
	GetHandleVerifier [0x00007FF767D22A03+1313971]
	GetHandleVerifier [0x00007FF767C1DD06+245686]
	(No symbol) [0x00007FF767BA758F]
	(No symbol) [0x00007FF767BA3804]
	(No symbol) [0x00007FF767BA3992]
	(No symbol) [0x00007FF767B9A3EF]
	BaseThreadInitThunk [0x00007FFA94047344+20]
	RtlUserThreadStart [0x00007FFA95B1CC91+33]


In [7]:
df = pd.read_csv("../outputs/Best_Buy/test_product_data.csv")
df_old = pd.read_csv("../outputs/Best_Buy/product_data.csv")


In [9]:
try:
    # See SKUs removed
    removed_skus = df_old[~df_old['SKU'].isin(df['SKU'])].copy()
    removed_skus['status'] = 'removed'
    removed_skus = removed_skus[['SKU', 'status']]


    # See SKUs added
    added_skus = df[~df['SKU'].isin(df_old['SKU'])].copy()
    added_skus['status'] = 'added'
    added_skus = added_skus[['SKU', 'status']]
    
    # Merge
    df_changed = pd.concat([removed_skus, added_skus], ignore_index=True)
    df_changed.to_csv('df_changed.csv', index=False)
except Exception as e:
    print("Not able to detect changes! Something went wrong: ",e)
finally:
    # Save the updated DataFrames to be new "old"
    df.to_csv("../outputs/Best_Buy/product_data.csv", index=False)
    
print(df.head(20))

                                                Link  \
0  https://www.bestbuy.com/site/ge-7-2-cu-ft-elec...   
1  https://www.bestbuy.com/site/ge-4-5-cu-ft-high...   
2  https://www.bestbuy.com/site/lg-4-5-cu-ft-he-s...   
3  https://www.bestbuy.com/site/whirlpool-5-0-cu-...   

                                                Name          SKU      Price  \
0  GE - 7.2 Cu. Ft. Electric Dryer with Long Vent...  gtd48easwwb    $579.99   
1  GE - 4.5 Cu. Ft. High-Efficiency Top Load Wash...  gtw485aswwb    $579.99   
2  LG - 4.5 Cu. Ft. HE Smart Front Load Washer an...   wkgx201hba  $1,999.99   
3  Whirlpool - 5.0 Cu. Ft. High Efficiency Stacka...    wfw8620hw  $1,019.99   

   Five Star  Review Amount  \
0        4.7             35   
1        4.5            193   
2        4.6            424   
3        4.2            159   

                                          Image Link  \
0  https://pisces.bbystatic.com/image2/BestBuy_US...   
1  https://pisces.bbystatic.com/image2/BestBuy_US.